# 🎯 ES診断ツール - Google Colab起動用

このノートブックを使用して、ES診断ツールをGoogle Colab上で起動します。

## 📋 実行手順

1. **セル1**: パッケージのインストール
2. **セル2**: GitHubリポジトリをクローン
3. **セル3**: CSVデータの準備（Google Drive / アップロード）
4. **セル4**: 前処理済みデータの確認・読み込み（オプション・高速化）
5. **セル5**: アプリケーションの起動
6. **セル6**: 前処理済みデータの保存（オプション・次回起動を高速化）

## ⚡ 高速起動について

**初回起動**: セル1〜5を順番に実行（3-5分）
**2回目以降**: セル6で保存した前処理済みデータを使用して数秒で起動！

---

## セル1: 必要なパッケージをインストール

In [ ]:
# 必要なパッケージをインストール
!pip install flask flask-cors pandas numpy scikit-learn tqdm -q

# sentence-transformers をインストール（セマンティック検索用）
print("🔧 sentence-transformersをインストール中...")
!pip install sentence-transformers -q
print("✅ sentence-transformersのインストールが完了しました")

# pyngrokを個別にインストール（タイムアウト対策）
!pip install pyngrok -q --default-timeout=100

print("✅ 全パッケージのインストールが完了しました")
print("\n📦 インストールされたパッケージ:")
print("  - Flask (Webアプリケーション)")
print("  - pandas (データ処理)")
print("  - numpy (数値計算)")
print("  - scikit-learn (TF-IDF、機械学習)")
print("  - sentence-transformers (セマンティック検索用BERT)")
print("  - tqdm (進捗表示)")
print("  - pyngrok (公開URL作成)")
print("\n⚡ パフォーマンス最適化:")
print("  - バッチ処理で高速化（40分 → 3-5分）")
print("  - GPU対応（T4有効時はさらに高速）")

## セル2: GitHubリポジトリをクローン

In [2]:
# リポジトリをクローン
!git clone https://github.com/ryosuke-fujii/es.git

# カレントディレクトリを変更
%cd es

print("✅ リポジトリのクローンが完了しました")
print("\n📁 ディレクトリ構造:")
!ls -la

Cloning into 'es'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 27 (delta 6), reused 26 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (27/27), 21.09 KiB | 2.64 MiB/s, done.
Resolving deltas: 100% (6/6), done.
/content/es
✅ リポジトリのクローンが完了しました

📁 ディレクトリ構造:
total 56
drwxr-xr-x 7 root root  4096 Nov  4 15:37 .
drwxr-xr-x 1 root root  4096 Nov  4 15:37 ..
-rw-r--r-- 1 root root 10402 Nov  4 15:37 COLAB_GUIDE.md
drwxr-xr-x 2 root root  4096 Nov  4 15:37 data
drwxr-xr-x 8 root root  4096 Nov  4 15:37 .git
-rw-r--r-- 1 root root   486 Nov  4 15:37 .gitignore
drwxr-xr-x 2 root root  4096 Nov  4 15:37 notebooks
-rw-r--r-- 1 root root  6790 Nov  4 15:37 README.md
-rw-r--r-- 1 root root    94 Nov  4 15:37 requirements.txt
drwxr-xr-x 2 root root  4096 Nov  4 15:37 src
drwxr-xr-x 2 root root  4096 Nov  4 15:37 templates


## セル3: CSVデータの準備

以下の2つの方法から選択してください：
- **方法A**: Google Driveから読み込む（推奨）
- **方法B**: 直接アップロード

In [ ]:
# ============================================
# 方法A: Google Driveから読み込む（推奨）
# ============================================

from google.colab import drive
import os

# Google Driveをマウント
drive.mount('/content/drive')

# 統合CSVファイルのパスを指定（就活会議 + ワンキャリア）
csv_path = "/content/drive/MyDrive/企画・マーケチーム/gaxi_自動化/Python/ESデータ整形/unified_es_data.csv"

# ファイルが存在するか確認
if os.path.exists(csv_path):
    print(f"✅ ファイルを発見: {csv_path}")
    # データソース情報を確認
    import pandas as pd
    df = pd.read_csv(csv_path)
    print(f"📊 データ件数: {len(df)}件")
    if 'data_source' in df.columns:
        print("\nデータソース内訳:")
        print(df['data_source'].value_counts())
else:
    print(f"❌ ファイルが見つかりません: {csv_path}")
    print("\n💡 ヒント: ファイルブラウザで目的のファイルを右クリック → 'パスをコピー'")

In [ ]:
# ============================================
# 方法B: 直接アップロード
# ============================================

from google.colab import files

# ファイルをアップロード
uploaded = files.upload()

# アップロードされたファイル名を取得
csv_path = list(uploaded.keys())[0]
print(f"✅ ファイルをアップロードしました: {csv_path}")

## セル5: データ読み込みとアプリケーション起動

In [ ]:
import threading
import time
import pickle
import numpy as np
from scipy import sparse
import sys
import os

# srcディレクトリをPythonパスに追加（重要！）
sys.path.insert(0, os.path.join(os.getcwd(), 'src'))

# アプリケーションをインポート
from app import app, load_csv_data, es_data, vectorizer, tfidf_matrix, sentence_model

# ============================================
# ngrokの認証トークンを設定
# ============================================
# https://dashboard.ngrok.com/ でトークンを取得してください

NGROK_TOKEN = "YOUR_NGROK_TOKEN"  # ここにあなたのngrokトークンを入力

# ngrokインポートとセットアップ（リトライ付き）
print("🔧 ngrokをセットアップ中...")
for attempt in range(3):
    try:
        from pyngrok import ngrok
        ngrok.set_auth_token(NGROK_TOKEN)
        print("✅ ngrok認証トークンを設定しました")
        break
    except Exception as e:
        if attempt < 2:
            print(f"⚠️ リトライ {attempt + 1}/3...")
            time.sleep(5)
        else:
            print(f"❌ ngrokのセットアップに失敗しました: {e}")
            print("\n💡 解決方法:")
            print("1. ランタイムを再起動してもう一度試す")
            print("2. セル1を再実行してパッケージを再インストール")
            raise

# ============================================
# データを読み込み（前処理済みデータまたは新規）
# ============================================

# USE_PREPROCESSEDが定義されていない場合のデフォルト値を設定
if 'USE_PREPROCESSED' not in globals():
    USE_PREPROCESSED = False
    preprocessed_files = {}
    preprocessed_dir = ""
    print("ℹ️ セル4がスキップされたため、前処理済みデータを使用しません")
    print("⏳ 新規データの読み込みを実行します（3-5分程度）\n")

if USE_PREPROCESSED:
    print("\n⚡ 前処理済みデータを読み込み中...")
    print("💡 これにより起動が大幅に高速化されます")
    
    # グローバル変数にアクセスするためにappモジュールから取得
    import app as app_module
    
    # es_dataを読み込み
    with open(preprocessed_files['es_data'], 'rb') as f:
        app_module.es_data = pickle.load(f)
    print(f"  ✅ es_data: {len(app_module.es_data)}件")
    
    # TF-IDF行列を読み込み
    app_module.tfidf_matrix = sparse.load_npz(preprocessed_files['tfidf_matrix'])
    print(f"  ✅ tfidf_matrix: {app_module.tfidf_matrix.shape}")
    
    # Vectorizerを読み込み
    with open(preprocessed_files['vectorizer'], 'rb') as f:
        app_module.vectorizer = pickle.load(f)
    print(f"  ✅ vectorizer")
    
    # セマンティックエンベディングを読み込み
    if os.path.exists(preprocessed_files['embeddings']):
        # Sentence-BERTモデルをロード（エンベディング計算には不要だが、モデルは必要）
        try:
            from sentence_transformers import SentenceTransformer
            if app_module.sentence_model is None:
                print("  📥 Sentence-BERTモデルをロード中...")
                app_module.sentence_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
                print("  ✅ モデルロード完了")
        except ImportError:
            print("  ⚠️ sentence-transformersが利用できません。セマンティック検索は無効です。")
        
        # エンベディングを読み込み
        app_module.es_data['semantic_embedding'] = list(np.load(preprocessed_files['embeddings']))
        print(f"  ✅ semantic_embeddings: {len(app_module.es_data['semantic_embedding'])}件")
    
    print("\n✅ 前処理済みデータの読み込みが完了しました（数秒で完了）")
    
else:
    print("\n📂 CSVデータを読み込み中...")
    print("⏳ 初回実行時はSentence-BERTモデルのダウンロードに数分かかります...")
    print("⚡ セマンティックエンベディング生成は3-5分程度で完了します（バッチ処理最適化済み）")
    print("")
    load_csv_data(csv_path)
    print("\n✅ データの読み込みが完了しました")

# ============================================
# Flaskアプリをバックグラウンドで起動
# ============================================
def run_flask():
    app.run(port=5000)

flask_thread = threading.Thread(target=run_flask, daemon=True)
flask_thread.start()

print("\n🚀 Flaskアプリを起動しました (ポート5000)")

# ============================================
# ngrokトンネルを作成
# ============================================
time.sleep(2)  # Flaskの起動を待つ

print("\n🌐 ngrokトンネルを作成中...")
try:
    public_url = ngrok.connect(5000)
    
    print("\n" + "="*60)
    print("🎉 アプリケーションが起動しました！")
    print("="*60)
    print(f"\n🌐 公開URL: {public_url}")
    print("\n💡 上記のURLをクリックしてアプリにアクセスしてください")
    print("\n⚠️ このノートブックを閉じるとアプリも停止します")
    print("\n🔍 搭載機能:")
    print("  - TF-IDFキーワードマッチング")
    print("  - セマンティック検索（BERT）で意味的類似性を理解")
    print("  - STAR構造分析でES品質を評価")
    
    if USE_PREPROCESSED:
        print("\n⚡ 前処理済みデータ使用:")
        print("  - 起動時間: 数秒（通常3-5分のところ）")
        print(f"  - データ保存先: {preprocessed_dir}")
    else:
        print("\n⚡ パフォーマンス:")
        print("  - バッチ処理最適化: エンベディング生成が40分→3-5分に短縮")
        print("  - 軽量モデル採用: 384次元（従来の768次元から高速化）")
        print("  - GPU自動検出: T4有効時は30秒〜1分でエンベディング生成")
        print(f"\n💾 次回起動用に前処理済みデータを保存するには、セル6を実行してください")
    
    print("="*60)
except Exception as e:
    print(f"\n❌ ngrokトンネルの作成に失敗しました: {e}")
    print("\n💡 トラブルシューティング:")
    print("1. NGROK_TOKENが正しく設定されているか確認")
    print("2. https://dashboard.ngrok.com/ でトークンを再確認")
    print("3. ランタイムを再起動して最初からやり直す")
    raise

## セル4: 前処理済みデータの確認・読み込み（オプション）

前回実行時に保存した前処理済みデータがあれば読み込み、なければ新規作成します。
これにより2回目以降の起動が高速化されます（3-5分 → 数秒）。

In [ ]:
import os
import sys

# srcディレクトリをPythonパスに追加
sys.path.insert(0, os.path.join(os.getcwd(), 'src'))

# ============================================
# 前処理済みデータのパス設定
# ============================================
# CSVと同じディレクトリに保存
csv_dir = os.path.dirname(csv_path)
preprocessed_dir = os.path.join(csv_dir, 'es_preprocessed_data')

# CSVファイル名からベース名を取得
csv_basename = os.path.splitext(os.path.basename(csv_path))[0]

# 前処理済みデータのファイルパス
preprocessed_files = {
    'es_data': os.path.join(preprocessed_dir, f'{csv_basename}_es_data.pkl'),
    'tfidf_matrix': os.path.join(preprocessed_dir, f'{csv_basename}_tfidf_matrix.npz'),
    'vectorizer': os.path.join(preprocessed_dir, f'{csv_basename}_vectorizer.pkl'),
    'embeddings': os.path.join(preprocessed_dir, f'{csv_basename}_embeddings.npy')
}

# 前処理済みデータが存在するかチェック
all_files_exist = all(os.path.exists(f) for f in preprocessed_files.values())

if all_files_exist:
    print("✅ 前処理済みデータを発見しました！")
    print(f"📂 保存先: {preprocessed_dir}")
    print("\n📁 利用可能なファイル:")
    for name, path in preprocessed_files.items():
        size_mb = os.path.getsize(path) / (1024 * 1024)
        print(f"  - {name}: {size_mb:.2f} MB")
    print("\n💡 これらのファイルを使用すると起動が高速化されます（数秒で完了）")
    USE_PREPROCESSED = True
else:
    print("ℹ️ 前処理済みデータが見つかりません")
    print("⏳ 初回起動のため、データの前処理を実行します（3-5分程度）")
    print(f"💾 完了後、前処理済みデータを以下に保存します:")
    print(f"   {preprocessed_dir}")
    USE_PREPROCESSED = False

## セル6: 前処理済みデータを保存（オプション）

**次回起動を高速化するため、前処理済みデータをGoogle Driveに保存します。**

このセルを実行すると：
- es_data（前処理済みDataFrame）
- TF-IDF行列とVectorizer
- セマンティックエンベディング

がCSVと同じディレクトリに保存され、次回起動時に自動的に読み込まれます。

⚠️ **注意**: セル5でアプリケーションを起動した後に実行してください。

In [ ]:
import os
import pickle
import numpy as np
from scipy import sparse

# appモジュールから前処理済みデータを取得
import app as app_module

# 前処理済みデータが存在するか確認
if app_module.es_data is None or len(app_module.es_data) == 0:
    print("❌ エラー: データが読み込まれていません")
    print("💡 先にセル5を実行してアプリケーションを起動してください")
else:
    print("💾 前処理済みデータを保存中...")
    print(f"📂 保存先: {preprocessed_dir}")
    
    # 保存先ディレクトリを作成
    os.makedirs(preprocessed_dir, exist_ok=True)
    
    # 1. es_dataを保存
    print("\n  💾 es_data を保存中...")
    with open(preprocessed_files['es_data'], 'wb') as f:
        pickle.dump(app_module.es_data, f)
    size_mb = os.path.getsize(preprocessed_files['es_data']) / (1024 * 1024)
    print(f"  ✅ es_data 保存完了: {size_mb:.2f} MB")
    
    # 2. TF-IDF行列を保存
    if app_module.tfidf_matrix is not None:
        print("\n  💾 tfidf_matrix を保存中...")
        sparse.save_npz(preprocessed_files['tfidf_matrix'], app_module.tfidf_matrix)
        size_mb = os.path.getsize(preprocessed_files['tfidf_matrix']) / (1024 * 1024)
        print(f"  ✅ tfidf_matrix 保存完了: {size_mb:.2f} MB")
    
    # 3. Vectorizerを保存
    if app_module.vectorizer is not None:
        print("\n  💾 vectorizer を保存中...")
        with open(preprocessed_files['vectorizer'], 'wb') as f:
            pickle.dump(app_module.vectorizer, f)
        size_mb = os.path.getsize(preprocessed_files['vectorizer']) / (1024 * 1024)
        print(f"  ✅ vectorizer 保存完了: {size_mb:.2f} MB")
    
    # 4. セマンティックエンベディングを保存
    if 'semantic_embedding' in app_module.es_data.columns:
        print("\n  💾 semantic_embeddings を保存中...")
        embeddings_array = np.array(app_module.es_data['semantic_embedding'].tolist())
        np.save(preprocessed_files['embeddings'], embeddings_array)
        size_mb = os.path.getsize(preprocessed_files['embeddings']) / (1024 * 1024)
        print(f"  ✅ semantic_embeddings 保存完了: {size_mb:.2f} MB")
    else:
        print("\n  ⚠️ semantic_embeddingが見つかりません（スキップ）")
    
    # 保存完了サマリー
    print("\n" + "="*60)
    print("✅ 前処理済みデータの保存が完了しました！")
    print("="*60)
    print(f"\n📂 保存先: {preprocessed_dir}")
    print("\n📁 保存されたファイル:")
    total_size = 0
    for name, path in preprocessed_files.items():
        if os.path.exists(path):
            size_mb = os.path.getsize(path) / (1024 * 1024)
            total_size += size_mb
            print(f"  ✅ {name}: {size_mb:.2f} MB")
    print(f"\n💾 合計サイズ: {total_size:.2f} MB")
    print("\n💡 次回起動時にこれらのファイルが自動的に読み込まれます")
    print("⚡ 起動時間が 3-5分 → 数秒 に短縮されます！")
    print("="*60)

## 🔧 トラブルシューティング

### 前処理済みデータが読み込まれない
```
前処理済みデータを保存したはずなのに認識されない
```

**対処法：**
- セル4で前処理済みデータの存在が確認されているか確認
- 保存先ディレクトリのパスが正しいか確認
- すべてのファイル（es_data.pkl, tfidf_matrix.npz, vectorizer.pkl, embeddings.npy）が揃っているか確認
- 再度セル6を実行してデータを保存し直す

### エンベディング生成が遅い（新しいバッチ処理版）
```
エンベディング生成に時間がかかる
```

**正常な処理時間:**
- **CPU環境**: 3-5分（35,000件の場合）
- **GPU環境（T4）**: 30秒〜1分
- 進捗バーが表示され、残り時間が確認できます

**処理時間の確認方法:**
- 実行時に「🧪 テスト: 最初の100件を処理...」で予想時間が表示されます
- tqdmの進捗バーで残り時間を確認できます

**さらに高速化したい場合:**
1. **前処理済みデータを使用**: セル6でデータを保存し、次回起動時に自動読み込み（数秒で起動）
2. Google ColabでGPU（T4）を有効化
   - ランタイム → ランタイムのタイプを変更 → T4 GPU を選択
3. データ量を減らす
   ```python
   # セル4の前に実行
   import pandas as pd
   df = pd.read_csv(csv_path)
   df = df.tail(10000)  # 最新1万件のみ
   df.to_csv('temp.csv', index=False)
   csv_path = 'temp.csv'
   ```

### sentence-transformersのインストールが遅い
```
初回インストール時に時間がかかる場合があります
```

**対処法：**
- sentence-transformersは大きなライブラリ（PyTorchを含む）のため、インストールに5-10分かかることがあります
- 進行中のメッセージが表示されていれば正常です
- 完了を待ってください

### Sentence-BERTモデルのダウンロードが遅い
```
初回実行時にモデルのダウンロードに時間がかかる
```

**対処法：**
- 初回実行時のみ、モデル（約120MB）がダウンロードされます
- 2回目以降はキャッシュされたモデルを使用するため高速です
- セル5で「📥 セマンティックモデルをダウンロード中...」と表示されていれば正常です

### 進捗バーが表示されない
```
tqdmの進捗バーが表示されない
```

**対処法：**
- Google Colabにはtqdmが標準で含まれていますが、表示されない場合は動作に問題ありません
- 「⚠️ tqdmがインストールされていません。進捗表示なしで実行します。」と表示されても処理は継続されます

### ngrokダウンロードのタイムアウトエラー
```
TimeoutError: The read operation timed out
```

**対処法：**
1. ランタイムを再起動して最初から実行
2. 別の時間帯に試す（ネットワーク混雑時を避ける）
3. セル1で個別にインストール：
   ```python
   !pip install pyngrok --default-timeout=100 -q
   ```

### CSVファイルが見つかりません
- セル3でファイルパスを確認してください
- Google Driveの場合、マウントが正しく行われているか確認してください
- ファイルブラウザで実際のパスをコピーして使用

### ngrok認証エラー
```
Invalid credentials
```

**対処法：**
- セル5の `NGROK_TOKEN` を正しく設定してください
- https://dashboard.ngrok.com/ でトークンを再確認
- トークンの前後にスペースや改行がないか確認

### メモリエラー
```
MemoryError
```

**対処法：**
- CSVファイルが大きすぎる可能性があります
- sentence-transformersも大量のメモリを使用します
- Google Colab Proの使用を検討
- データをフィルタリング：
   ```python
   import pandas as pd
   df = pd.read_csv(csv_path)
   df = df.tail(10000)  # 最新1万件のみ
   df.to_csv('temp.csv', index=False)
   csv_path = 'temp.csv'
   ```

### インポートエラー
```
ModuleNotFoundError: No module named 'app'
```

**対処法：**
- セル2でリポジトリのクローンが成功しているか確認
- `%cd es` が実行されているか確認
- セル5を再実行

### セマンティック検索が使えない場合
```
⚠️ sentence-transformersがインストールされていません。TF-IDFのみを使用します。
```

**対処法：**
- アプリは動作しますが、意味的類似性は使われません
- セル1を再実行してsentence-transformersをインストール
- ランタイムを再起動してすべてのセルを順番に実行

---

## ⚡ パフォーマンス改善の詳細

### 最適化内容
1. **前処理済みデータの活用**: 初回起動後にデータを保存し、次回は数秒で起動
2. **バッチ処理**: 1件ずつ処理 → 32件ずつバッチ処理
3. **軽量モデル**: 768次元 → 384次元（精度はほぼ同じ、速度2倍）
4. **GPU対応**: CUDA利用可能時は自動的にGPU使用
5. **進捗表示**: tqdmで残り時間を可視化

### 処理時間の比較（35,000件の場合）
- **旧実装（apply）**: 約40分
- **新実装（CPU・初回）**: 3-5分（約8-13倍高速化）
- **新実装（GPU T4・初回）**: 30秒〜1分（約40-80倍高速化）
- **前処理済みデータ使用（2回目以降）**: 数秒（約400-800倍高速化）

### 前処理済みデータのサイズ目安
- **es_data.pkl**: 約50-100 MB（データ件数による）
- **tfidf_matrix.npz**: 約20-50 MB
- **vectorizer.pkl**: 約5-10 MB
- **embeddings.npy**: 約50-150 MB
- **合計**: 約125-310 MB（35,000件の場合）